In [1]:
import os
from estnltk import Text
from estnltk_neural.taggers import StanzaSyntaxTagger
import pandas as pd
import copy
import numpy as np
import sklearn
import csv
import sqlite3
from estnltk.converters import text_to_json
import json

Fraasid, mis on ühe võrra suuremad mustrist.

In [2]:
# patterns are kept in a list of strings
patterns = None

with open('correct_tree_patterns.txt', 'r') as f:
    patterns = [line.rstrip() for line in f]

for pattern in patterns:
    print(pattern)

1 PER 0 root H,2 OTHER 1 flat H,3 OTHER 1 flat H
1 LOC 2 nmod G,2 OTHER 0 root S
1 LOC 2 nmod H,2 OTHER 0 root D
1 LOC 2 nmod H,2 OTHER 0 root H
1 LOC 2 nmod H,2 OTHER 0 root S
1 ORG 2 nmod S,2 OTHER 0 root S
1 ORG 2 nmod Y,2 OTHER 0 root S
1 LOC 2 amod G,2 OTHER 0 root S
1 LOC 2 amod H,2 OTHER 0 root S
1 LOC 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat H
1 ORG 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat S
1 LOC 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat Y
1 LOC 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos S,4 OTHER 3 flat H
1 ORG 2 nmod S,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat H
1 ORG 2 nmod S,2 OTHER 0 root S,3 OTHER 2 appos S,4 OTHER 3 flat H
1 LOC 2 nmod Y,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat H
1 LOC 2 obl H,2 OTHER 3 acl A,3 OTHER 0 root S
1 PER 2 nmod H,2 OTHER 0 root S,3 OTHER 2 nmod S
1 LOC 2 obl A,2 OTHER 3 acl A,3 OTHER 0 root S
1 LOC 2 obl Y,2 OTHER 3 acl A,3 OTHER 0 root S
1 OTHER 3 advmod D|I,2 PER 3 nmod H,

In [3]:
patterns_simplified = []

for pattern in patterns:
    pieces = [wrd.split() for wrd in pattern.split(',')]
    for piece in pieces:
        piece.pop(1)

    for i in range(len(pieces)):
        pieces[i] = ' '.join(pieces[i])
    patterns_simplified.append(','.join(pieces))
    
for pattern in patterns_simplified:
    print(pattern)

1 0 root H,2 1 flat H,3 1 flat H
1 2 nmod G,2 0 root S
1 2 nmod H,2 0 root D
1 2 nmod H,2 0 root H
1 2 nmod H,2 0 root S
1 2 nmod S,2 0 root S
1 2 nmod Y,2 0 root S
1 2 amod G,2 0 root S
1 2 amod H,2 0 root S
1 2 nmod H,2 0 root S,3 2 appos H,4 3 flat H
1 2 nmod H,2 0 root S,3 2 appos H,4 3 flat S
1 2 nmod H,2 0 root S,3 2 appos H,4 3 flat Y
1 2 nmod H,2 0 root S,3 2 appos S,4 3 flat H
1 2 nmod S,2 0 root S,3 2 appos H,4 3 flat H
1 2 nmod S,2 0 root S,3 2 appos S,4 3 flat H
1 2 nmod Y,2 0 root S,3 2 appos H,4 3 flat H
1 2 obl H,2 3 acl A,3 0 root S
1 2 nmod H,2 0 root S,3 2 nmod S
1 2 obl A,2 3 acl A,3 0 root S
1 2 obl Y,2 3 acl A,3 0 root S
1 3 advmod D|I,2 3 nmod H,3 0 root S
1 3 advmod D,2 3 nmod H,3 0 root S
1 3 advmod S,2 3 nmod H,3 0 root S
1 3 amod H,2 3 amod A,3 0 root S
1 3 amod H,2 3 amod O,3 0 root S
1 3 amod A,2 3 nmod G,3 0 root S
1 3 amod A,2 3 nmod H,3 0 root S
1 3 amod A,2 3 nmod Y,3 0 root S
1 3 amod O,2 3 nmod H,3 0 root S
1 3 det P|N,2 3 nmod H,3 0 root S
1 3 det A,2

In [4]:
patterns_more_simplified = []

for pattern in patterns_simplified:
    pieces = [wrd.split() for wrd in pattern.split(',')]
    for piece in pieces:
        piece.pop(3)

    for i in range(len(pieces)):
        pieces[i] = ' '.join(pieces[i])
    patterns_more_simplified.append(','.join(pieces))
    
for pattern in patterns_more_simplified:
    print(pattern)

1 0 root,2 1 flat,3 1 flat
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 amod,2 0 root
1 2 amod,2 0 root
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 obl,2 3 acl,3 0 root
1 2 nmod,2 0 root,3 2 nmod
1 2 obl,2 3 acl,3 0 root
1 2 obl,2 3 acl,3 0 root
1 3 advmod,2 3 nmod,3 0 root
1 3 advmod,2 3 nmod,3 0 root
1 3 advmod,2 3 nmod,3 0 root
1 3 amod,2 3 amod,3 0 root
1 3 amod,2 3 amod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 det,2 3 nmod,3 0 root
1 3 det,2 3 nmod,3 0 root
1 3 det,2 3 nmod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nm

In [39]:
patterns_with_ner_timex_other = []

for pattern in patterns:
    pieces = [wrd.split() for wrd in pattern.split(',')]
    for piece in pieces:
        piece.pop(4)

    for i in range(len(pieces)):
        pieces[i] = ' '.join(pieces[i])
    patterns_with_ner_timex_other.append(','.join(pieces))
    
for pattern in patterns_with_ner_timex_other:
    print(pattern)

1 PER 0 root,2 OTHER 1 flat,3 OTHER 1 flat
1 LOC 2 nmod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root
1 ORG 2 nmod,2 OTHER 0 root
1 ORG 2 nmod,2 OTHER 0 root
1 LOC 2 amod,2 OTHER 0 root
1 LOC 2 amod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 ORG 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 ORG 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 ORG 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 obl,2 OTHER 3 acl,3 OTHER 0 root
1 PER 2 nmod,2 OTHER 0 root,3 OTHER 2 nmod
1 LOC 2 obl,2 OTHER 3 acl,3 OTHER 0 root
1 LOC 2 obl,2 OTHER 3 acl,3 OTHER 0 root
1 OTHER 3 advmod,2 PER 3 nmod,3 OTHER 0 root
1 OTHER 3 advmod,2 LOC 3 nmod,3 OTHER 0 root
1 OTHER 3 advmod,2 LOC 3 nmod,3 OTHER 0 root
1 LOC 3 amod,2 OTHE

In [5]:
df = pd.read_pickle('phrase_examples_filtered_sub10000/non_atomic_phrases.pkl')

In [6]:
df

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,...,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex,pos_sequence_verb_info,graph_verb_info,graph_code_verb_info,graph_code_pos_verb_info,graph_code_pos_ner_timex_verb_info
0,Text(text='neist teede äärtesse'),3,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(84, 104)",0,...,P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))",P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))"
1,Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),9,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nsubj_cop_phrase,"(0, 76)",1,...,H-S-S-S-S-S-S-H-H,"(1, 2, 3, 4, 5, 6, 7, 0, 8, 9)","((0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(0-ner_ORG-ner_ORG-0-0-0-0-0-0,S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))",H-S-S-S-S-S-S-H-H,"(1, 2, 3, 4, 5, 6, 7, 0, 8, 9)","((0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(0-ner_ORG-ner_ORG-0-0-0-0-0-0,S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))"
2,Text(text='Tartu ülikooli ajaloo'),3,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 21)",1,...,H-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, nmod),(3, 2, nmod))","(S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))","(0-ner_ORG-ner_ORG,S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))",H-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, nmod),(3, 2, nmod))","(S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))","(0-ner_ORG-ner_ORG,S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))"
3,Text(text='Tartu ülikooli ajaloo osakonna'),4,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 30)",1,...,H-S-S-S,"(1, 2, 3, 4, 0)","((0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(0-ner_ORG-ner_ORG-0,S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))",H-S-S-S,"(1, 2, 3, 4, 0)","((0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(0-ner_ORG-ner_ORG-0,S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))"
4,Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli'),6,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 50)",1,...,H-S-S-S-S-S,"(1, 2, 3, 4, 5, 6, 0)","((0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(0-ner_ORG-ner_ORG-0-0-0,S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))",H-S-S-S-S-S,"(1, 2, 3, 4, 5, 6, 0)","((0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(0-ner_ORG-ner_ORG-0-0-0,S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051,Text(text='konkureeriva kaabeltelevisooni firma'),3,2024-03-10T01:35,1526710,61973,"(522, 718)",aja_

In [15]:
test = []

for idx, row in df.iterrows():
    test.append(row['phrase'])
    if idx == 5:
        break
    
test

[Text(text='neist teede äärtesse'),
 Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),
 Text(text='Tartu ülikooli ajaloo'),
 Text(text='Tartu ülikooli ajaloo osakonna'),
 Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli'),
 Text(text='kõik Hollywoodi või ukraina filmid , aga eesti filmid ilmtingimata')]

In [26]:
def get_ner_timex(text_obj, stanza_word):
    ner = None
    nertag = None
    if len(text_obj.ner) > 0:
        word = text_obj.words.get(stanza_word)
        for n in text_obj.ner:
            for part in n:
                if part==word:
                    ner=word
                    nertag=n.nertag
    timex = text_obj.timexes.get(stanza_word)
    if ner:
        return nertag
    if timex:
        return 'TIMEX'
    return 'OTHER'

In [27]:
def get_ner(text_obj, stanza_word):
    ner = None
    nertag = None
    if len(text_obj.ner) > 0:
        word = text_obj.words.get(stanza_word)
        for n in text_obj.ner:
            for part in n:
                if part==word:
                    ner=word
                    nertag=n.nertag
    if ner:
        return nertag
    return 'OTHER'

In [28]:
def get_timex(text_obj, stanza_word):
    timex = text_obj.timexes.get(stanza_word)
    if timex:
        return 'TIMEX'
    return 'OTHER'

In [29]:
def get_unique_POS(word):
    #pos_list = word.morph_analysis['partofspeech']
    infinite_verb_forms = ['da', 'des', 'ma', 'maks', 'mas', 'mast', 'mata', 'nud', 'tav', 'tud', 'v']
    # if POS is ambiguous, only unique tags are kept, e.g. ['V', 'A', 'A'] -> ['V', 'A']
    pos_list = []
    for i in range(len(word.morph_analysis['partofspeech'])):
        if word.morph_analysis['partofspeech'][i] == 'V':
            if word.morph_analysis['form'][i] in infinite_verb_forms:
                pos_list.append('V_inf')
            elif word.form[i] == 'neg':
                pos_list.append('V_neg')
            else:
                pos_list.append('V_fin')
        else:
            pos_list.append(word.morph_analysis['partofspeech'][i])
    
    if len(pos_list) > 1:
        char_unique = [char for indx, char in enumerate(pos_list) if char not in pos_list[:indx]]
        if len(char_unique) < 2:
            return char_unique[0]
        return '|'.join(char_unique)
    return pos_list[0]

In [33]:
def get_word_form(word):
    pieces = word.morph_analysis['form'][0].split()
    if len(pieces) < 1:
        return '0'
    else:
        if len(pieces) > 1:
            return pieces[1]
        else:
            return pieces[0]

In [21]:
def find_patterns(phrase):
    pattern_matches_found = []
    
    for i in range(len(phrase.stanza_syntax) - 1):
        pattern_members = []
        ids = []

        pattern_members.append(phrase.stanza_syntax[i])
        ids.append([phrase.stanza_syntax[i].id, phrase.stanza_syntax[i].head])

        for j in range(i + 1, len(phrase.stanza_syntax)):
            pattern_found = []
            for k in range(len(pattern_members)):
                if phrase.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in phrase.stanza_syntax[j].children or phrase.stanza_syntax[j].parent != None and phrase.stanza_syntax[j].parent == pattern_members[k].parent:
                    pattern_members.append(phrase.stanza_syntax[j])
                    ids.append([phrase.stanza_syntax[j].id, phrase.stanza_syntax[j].head])
        
            ids_for_pattern = copy.deepcopy(ids)
            for l in range(len(ids_for_pattern)):
                temp = ids_for_pattern[l][0]
                ids_for_pattern[l][0] = l+1
                for m in range(len(ids)):
                    if ids[m][1] == temp:
                        ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
            word_ids = [word_id[0] for word_id in ids_for_pattern]
            for l in range(len(ids_for_pattern)):
                if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                    ids_for_pattern[l][1] = 0
                elif ids_for_pattern[l][1] not in word_ids:
                    ids_for_pattern[l][1] = 0
            
            for l in range(len(pattern_members)):
                deprel = pattern_members[l].deprel
                if ids_for_pattern[l][1] == 0 and deprel != 'root':
                    deprel = 'root'
                pattern_found.append([str(ids_for_pattern[l][0]), str(ids_for_pattern[l][1]), deprel])
                    
            sub_pattern = [" ".join(word_info) for word_info in pattern_found]
            
            for l in range(len(patterns_more_simplified)):
                if patterns_more_simplified[l] in ",".join(sub_pattern) and len(sub_pattern) == len(patterns_more_simplified[l].split(',')) + 1:
                    pattern_matches_found.append([patterns_more_simplified[l], ",".join(sub_pattern), " ".join([member.text for member in pattern_members]), [member for member in pattern_members]])

            if j == len(phrase.stanza_syntax) - 1:
                pattern_members = []
                ids = []
                break             
                    
    return pattern_matches_found

In [22]:
for text in test:
    print(find_patterns(text))

[]
[['1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 appos', 'õppetooli vanemlaborant Sirje', [Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]], ['1 2 nmod,2 0 root', '1 2 nmod,2 0 root,3 2 appos', 'õppetooli vanemlaborant Sirje', [Span('õppetooli', [{'id': 6, 

In [23]:
matches = {'atomic_pattern': [], 'pattern_tree': [], 'phrase_text': [], 'phrase_spans': [], 'parent_phrase': []}

for idx, row in df.iterrows():
    m = find_patterns(row['phrase'])
    if len(m) > 0:
        for match in m:
            matches['atomic_pattern'].append(match[0])
            matches['pattern_tree'].append(match[1])
            matches['phrase_text'].append(match[2])
            matches['phrase_spans'].append(match[3])
            matches['parent_phrase'].append(row['phrase'])
            
matches_df = pd.DataFrame(matches)

In [24]:
matches_df

,atomic_pattern,pattern_tree,phrase_text,phrase_spans,parent_phrase
0,"1 2 nmod,2 0 root","1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
1,"1 2 nmod,2 0 root","1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,"1 2 nmod,2 0 root","1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,"1 2 nmod,2 0 root","1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,"1 2 nmod,2 0 root","1 2 nmod,2 0 root,3 2 appos",õppetooli vaneml

In [30]:
# NER/TIMEX/OTHER
combinations = []
for idx, row in matches_df.iterrows():
    combination = []
    for stanza_word in row['phrase_spans']:
        combination.append(get_ner_timex(row['parent_phrase'], stanza_word))
    combinations.append("-".join(combination))

matches_df.insert(0, 'ner/timex/other', combinations)

In [31]:
# POS-sequences
sequences = []
for idx, row in matches_df.iterrows():
    sequence = []
    for stanza_word in row['phrase_spans']:
        sequence.append(get_unique_POS(stanza_word))
    sequences.append("-".join(sequence))

matches_df.insert(2, 'pattern_pos', sequences)

In [34]:
# word forms
forms = []
for idx, row in matches_df.iterrows():
    form = []
    for stanza_word in row['phrase_spans']:
        form.append(get_word_form(stanza_word))
    forms.append("-".join(form))

matches_df.insert(3, 'pattern_form', forms)

In [35]:
# NER
combinations_ner = []
for idx, row in matches_df.iterrows():
    combination_ner = []
    for stanza_word in row['phrase_spans']:
        combination_ner.append(get_ner(row['parent_phrase'], stanza_word))
    combinations_ner.append("-".join(combination_ner))

matches_df.insert(4, 'pattern_ner', combinations_ner)

In [36]:
# TIMEX
combinations_timex = []
for idx, row in matches_df.iterrows():
    combination_timex = []
    for stanza_word in row['phrase_spans']:
        combination_timex.append(get_timex(row['parent_phrase'], stanza_word))
    combinations_timex.append("-".join(combination_timex))

matches_df.insert(5, 'pattern_timex', combinations_timex)

In [37]:
matches_df

,ner/timex/other,atomic_pattern,pattern_pos,pattern_form,pattern_ner,pattern_timex,pattern_tree,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
1,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict(

In [44]:
# adding information about pattern status (with ner/timex) -> 0 (pattern does not exist in pattern list), 
# 1 (pattern exists in pattern list)
pattern_statuses = []
for idx, row in matches_df.iterrows():
    ner_timex = row['ner/timex/other'].split('-')
    pattern_splitted = [part.split() for part in row['atomic_pattern'].split(',')]
    for i in range(len(pattern_splitted)):
        pattern_splitted[i].insert(1, ner_timex[i])
    for i in range(len(pattern_splitted)):
        pattern_splitted[i] = " ".join(pattern_splitted[i])
    new_pattern = ",".join(pattern_splitted)
    
    if new_pattern in patterns_with_ner_timex_other:
        pattern_statuses.append(1)
    else:
        pattern_statuses.append(0)

matches_df.insert(6, 'pattern_status_ner_timex_other', pattern_statuses)


In [43]:
matches_df

,ner/timex/other,atomic_pattern,pattern_pos,pattern_form,pattern_ner,pattern_timex,pattern_tree,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
1,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root,3 2 appos",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict(

In [48]:
# adding column with start and end positions of phrases
phrase_startends = []
for idx, row in matches_df.iterrows():
    start = row['phrase_spans'][0].start
    end = row['phrase_spans'][-1].end
    phrase_startends.append([start, end])
    
matches_df.insert(8, 'startend', phrase_startends)

In [49]:
matches_df

,ner/timex/other,atomic_pattern,pattern_pos,pattern_form,pattern_ner,pattern_timex,pattern_status_ner_timex_other,pattern_tree,startend,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
1,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span

In [50]:
unique_parent_phrases = []

for idx, row in matches_df.iterrows():
    if row['parent_phrase'] not in unique_parent_phrases:
        unique_parent_phrases.append(row['parent_phrase'])

In [51]:
len(unique_parent_phrases)

755

In [52]:
# parent phrase IDs
parent_phrase_ids = []
for idx, row in matches_df.iterrows():
    for i in range(len(unique_parent_phrases)):
        if unique_parent_phrases[i] == row['parent_phrase']:
            parent_phrase_ids.append(i)

matches_df.insert(12, 'parent_phrase_id', parent_phrase_ids)

In [53]:
matches_df

,ner/timex/other,atomic_pattern,pattern_pos,pattern_form,pattern_ner,pattern_timex,pattern_status_ner_timex_other,pattern_tree,startend,phrase_text,phrase_spans,parent_phrase,parent_phrase_id
0,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),0
1,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),0
2,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}]), Span('Sirje', [{'id': 8, 'lemma': 'Sirje', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 7, 'deprel': 'appos', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),0
3,OTHER-OTHER-OTHER,"1 2 nmod,2 0 root",S-S-H,g-n-n,OTHER-OTHER-OTHER,OTHER-OTHER-OTHER,0,"1 2 nmod,2 0 root,3 2 appos","[41, 70]",õppetooli vanemlaborant Sirje,"[Span('õppetooli', [{'id': 6, 'lemma': 'õppetool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 7, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('vanemlaborant', [{'id': 7, 'lemma': 'vanemlaborant', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <

In [54]:
con = sqlite3.connect("correct_noun_phrases_2.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE correct_phrase_patterns_2(ID INTEGER PRIMARY KEY, ner_timex_other TEXT, atomic_pattern TEXT, pattern_pos TEXT, pattern_form TEXT, pattern_ner TEXT, pattern_timex TEXT, pattern_status_ner_timex_other INTEGER, pattern_tree TEXT, startend TEXT, phrase_text TEXT, phrase_word_list TEXT, parent_phrase_id INTEGER)")

In [57]:
con.close()

In [58]:
con = sqlite3.connect("correct_noun_phrases_2.db")
cur = con.cursor()

for idx, row in matches_df.iterrows():
    phrase_word_list = row['phrase_text'].split()
    phrase_word_list_json = json.dumps(phrase_word_list)
    #parent_phrase_json = text_to_json(row['parent_phrase'])
    startend_str = str(row['startend'])
    cur.execute("""INSERT INTO correct_phrase_patterns_2
                            (ner_timex_other, atomic_pattern, pattern_pos, pattern_form, pattern_ner, pattern_timex, pattern_status_ner_timex_other, pattern_tree, startend, phrase_text, phrase_word_list, parent_phrase_id)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);""", (row['ner/timex/other'], row['atomic_pattern'], row['pattern_pos'], row['pattern_form'], row['pattern_ner'], row['pattern_timex'], row['pattern_status_ner_timex_other'], row['pattern_tree'], startend_str, row['phrase_text'], phrase_word_list_json, row['parent_phrase_id']))
    
    con.commit()

con.close()

In [59]:
con = sqlite3.connect("correct_parent_phrases_2.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE correct_parent_phrases_2(ID INTEGER PRIMARY KEY, phrase_id INTEGER, phrase TEXT)")

In [60]:
con.close()

In [61]:
con = sqlite3.connect("correct_parent_phrases_2.db")
cur = con.cursor()

for i in range(len(unique_parent_phrases)):
    parent_phrase_json = text_to_json(unique_parent_phrases[i])
    cur.execute("""INSERT INTO correct_parent_phrases_2
                            (phrase_id, phrase)
                            VALUES (?, ?);""", (i, parent_phrase_json))
    
    con.commit()

con.close()